In [1]:
import googlemaps
import time 
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import requests
from urllib.parse import urlencode


In [97]:
data  = pd.read_csv('data/Datos.csv',index_col=0)

In [167]:
data.head()

,estacion,Estado_nieve,Estado,pistas_azules,pistas_negras,pistas_rojas,pistas_verdes,n_pistas,remontes,Snowpark,KM disponibles,KM totales,precio_adultos €,Espesor_nieve cm,lat,long
0,Javalambre,Dura,Abierta,9,0.0,1.0,5,18,9,1,4.00,15.00,36.0,110.0,40.112247,-1.032631
1,Punto de Nieve Santa Inés,Polvo,Abierta,1,0.0,0.0,1,2,2,1,1.80,1.80,20.0,90.0,42.038054,-2.785251
2,Valdelinares,Polvo/Dura,Abierta,4,0.0,4.0,6,14,12,1,15.00,17.40,37.5,110.0,40.379822,-0.632447
3,Valdezcaray,desconocido,Abierta,6,2.0,10.0,6,26,9,0,9.55,22.35,28.5,110.0,42.257502,-2.970729
4,Sierra Nevada,Húmeda,Abierta,49,7.0,54.0,18,135,21,1,9.84,111.40,54.0,40.0,37.092935,-3.399549


In [2]:
load_dotenv()

True

In [5]:
token = os.getenv('ApiGooglekey')

In [12]:
cliente = googlemaps.Client(key = token)


In [185]:
prueba = cliente.places_nearby(location = '40.11224741452416,-1.0326308232622217', radius= 4000, open_now = False, type = 'restaurant')


In [186]:
def restaurantfinder(lat, long):
    re = cliente.places_nearby(location = f'{lat},{long}', radius= 5000, open_now = False, type = 'restaurant')
    return re

In [225]:
cord[5]
restaurantfinder(42.80986840508656,-0.5085355682272364)
#df.assign(x=lambda df: df.coord.map(lambda c: c[0]))
#df.assign(y=lambda df: df.coord.map(lambda c: c[1]))

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 42.7881174, 'lng': -0.5290718999999999},
    'viewport': {'northeast': {'lat': 42.7894663802915,
      'lng': -0.5277229197084979},
     'southwest': {'lat': 42.7867684197085, 'lng': -0.530420880291502}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'name': 'Hotel Tobazo - Candanchú',
   'photos': [{'height': 957,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117263731997351463314">Hotel Tobazo - Candanchú</a>'],
     'photo_reference': 'Aap_uEAtobzr8Z8aAFd_-QshOcHDAvXspGYGTJ31RjOodhrIzgm-0PpwsSKUWAbTECHvhwD1vF00eDDZUQBDjE6nhclsxyrOt7vOfQOIUMBVpA61nAQeW9p89-5ypU-ST_KK14r90a4nfhkVOkzzhu4yYBATbDcG63d8irbCgMW9PW1Dfct4',
     'width': 960}],
   'place_id': 'ChIJ5-ljiqecVw0RNii

In [210]:
#prueba de sonia
#nombres = data.estacion
#lat = data.lat
#long = data.long
#cord = list(zip(lat,long))
respuesta = []
restauranteasociado = data['estacion']
for i,row in data.iterrows():
    respuesta.append(restaurantfinder(row.lat,row.long))
    #restauranteasociado.append(data["estacion"])
#restauranteasociado

In [229]:
print(len(respuesta))

31


In [218]:
# Sonia's ideas ideitas
megalista_restaurantes =[]
for i in range (len(respuesta)):
    estacion = restauranteasociado[i]
    for i in range(len(respuesta)):
        new_dict = {'nombre_est':estacion, 'coord':None, 'nombre_rest':None,'adress':None,'rating':None}
        #para las coordenadas
        try:
            new_dict["coord"] = [respuesta[i]['results'][0]['geometry']['location']['lat'],respuesta[i]['results'][0]['geometry']['location']['lng']]

        except:
            try:
                new_dict["coord"] =[respuesta[i]['results'][0]['viewport']['northeast']['lat'],respuesta[i]['results'][0]['viewport']['northeast']['lng']]
            except:
                pass
        #para el nombre
        try:
            new_dict['nombre_rest'] = [respuesta[i]['results'][0]['name']]
        except:
            pass
        #para adress
        try:
            new_dict['adress'] = respuesta[i]['results'][0]['vicinity']
        except:
            pass
        #rating
        try:
            new_dict['rating'] = respuesta[i]['results'][0]['rating']
        except:
            pass
        #por ultimo appendeamos el diccionario a un megalista
        megalista_restaurantes.append(new_dict)
        

In [231]:
df = pd.DataFrame(megalista_restaurantes)
df.adress.value_counts()

Carretera General Puerto Braña-San Isidro, s/n, La Raya                   62
km 40, M-604, Puerto de Cotos                                             62
Plaza Iglesia, 8, Valdelinares                                            31
Urbanización Pla de L'Ermita, Carrer Sant Quirze, s/n, Pla de l'Ermita    31
Unnamed Road 40592, Cerezo de Arriba                                      31
Calle Obispo, 11, Cerler                                                  31
Carretera de Baqueira a Beret s/n Cota 1700, Baqueira                     31
Carretera d'Espot a Sant Maurici, 11, Espot                               31
Pla de Morens, SN, Setcases                                               31
Calle Mayor, 57, Candelario                                               31
Calle San Miguel, 4, Panticosa                                            31
Passeig Agnès Fabra, 4, Alp                                               31
CTRA.ASTUN-CANDANCHU SN, Canfranc                                         31

In [174]:

nombres = data.estacion
lat = data.lat
long = data.long
cord = list(zip(lat,long))
respuesta = []
pistaasociada = []
for i in cord:
    respuesta.append(restaurantfinder(i[0],i[1]))
    pistaasociada.append()

TypeError: append() takes exactly one argument (0 given)

In [163]:
d = pd.DataFrame([nombres,])

In [164]:
respuesta[15]

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}

In [165]:
latitud = []
longitud = []
for i in range(len(respuesta)):
    try:
        latitud.append(respuesta[i]['results'][0]['geometry']['location']['lat'])
        longitud.append(respuesta[i]['results'][0]['geometry']['location']['lng'])
    except:
        try:
            latitud.append(respuesta[i]['results'][0]['viewport']['northeast']['lat'])
            longitud.append(respuesta[i]['results'][0]['viewport']['northeast']['lng'])
        except:
            latitud.append(None)
            longitud.append(None)

In [144]:
respuesta[1]['results'][0]['geometry']['location']['lat']

42.7028647

In [122]:
respuesta[0]['results'][0]['geometry']['location']['lng']

-1.029947

In [90]:
cord = []
for i in respuesta:
    for k,v in i.items():
        if k =='results':
            if v == 'geometry':
                cord.append(value.get('location'))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [24]:
respuesta

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 40.1098552, 'lng': -1.029947},
    'viewport': {'northeast': {'lat': 40.1112041802915,
      'lng': -1.028598019708498},
     'southwest': {'lat': 40.1085062197085, 'lng': -1.031295980291502}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
   'icon_background_color': '#FF9E67',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
   'name': 'el portillo BURGERBAR',
   'place_id': 'ChIJJxZSeE8bXg0R8A9O65HgDbM',
   'plus_code': {'compound_code': '4X5C+W2 Camarena de la Sierra, Spain',
    'global_code': '8CGW4X5C+W2'},
   'rating': 5,
   'reference': 'ChIJJxZSeE8bXg0R8A9O65HgDbM',
   'scope': 'GOOGLE',
   'types': ['restaurant', 'food', 'point_of_interest', 'establishment'],
   'user_ratings_total': 1,
   'vicinity': 'Camarena de la Sierra, Unnamed Road, 44459'}],
 'status': 'OK'}